In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
Ts = 501
segments = 12 # beads = 13

In [4]:
f = []; m1 = []; m2 = []

for i in range(Ts):
    f_ = pd.read_csv('data5/Filament_5s133R0.8Ts'+str(i)+'.csv', names=['x','y','z'], skiprows=1)
    f.append(f_)
    m1_ = pd.read_csv('data5/Specie1_5s133R0.8Ts'+str(i)+'.csv', names=['x','y','z'], skiprows=1)
    m1.append(m1_)
    m2_ = pd.read_csv('data5/Specie2_5s133R0.8Ts'+str(i)+'.csv', names=['x','y','z'], skiprows=1)
    m2.append(m2_)

In [389]:
m1all = m1[0].drop(['z'], axis=1)
m2all = m2[0].drop(['z'], axis=1)

for i in range(Ts-1):
    m1all = m1all.append(m1[i+1].drop(['z'], axis=1))
    m2all = m2all.append(m2[i+1].drop(['z'], axis=1))

In [440]:
# Identify the time step
count = 0
Ts_count = 0
Ts_no = []

for i in m1all.index == 0:
    if i == True:
        count+=1
        Ts_no.append(Ts_count)
        Ts_count+=1
    else:
        Ts_no.append(Ts_count-1)

In [413]:
np.sum(1*m1all.duplicated())

8951

In [417]:
m1_dups = 1*m1all.duplicated()

In [426]:
m1_dups_df = pd.DataFrame({'dup':m1_dups})
m1_dups_df

,dup
0,0
1,0
2,0
3,0
4,0
...,...
33,0
34,0
35,0
36,0


In [636]:
Tsno = pd.DataFrame({'Ts':Ts_no})
#Tsno

In [437]:
m1all_dups = pd.concat([m1all,m1_dups_df], axis=1)

In [438]:
m1all_dups

,x,y,dup
0,3.56669,-0.00584,0
1,3.69861,0.00178,0
2,2.48554,0.00002,0
3,1.80410,-0.01313,0
4,3.35026,0.00422,0
...,...,...,...
33,2.97887,-1.46184,0
34,2.73518,-1.57747,0
35,2.74984,-1.58911,0
36,2.73792,-1.57618,0


In [461]:
m1all_dups_ = m1all_dups.reset_index(drop=True)
m1all_dups_

,x,y,dup
0,3.56669,-0.00584,0
1,3.69861,0.00178,0
2,2.48554,0.00002,0
3,1.80410,-0.01313,0
4,3.35026,0.00422,0
...,...,...,...
17589,2.97887,-1.46184,0
17590,2.73518,-1.57747,0
17591,2.74984,-1.58911,0
17592,2.73792,-1.57618,0


In [537]:
m1r08 = pd.concat([Tsno,m1all_dups_],axis=1)
m1r08

,Ts,x,y,dup
0,0,3.56669,-0.00584,0
1,0,3.69861,0.00178,0
2,0,2.48554,0.00002,0
3,0,1.80410,-0.01313,0
4,0,3.35026,0.00422,0
...,...,...,...,...
17589,500,2.97887,-1.46184,0
17590,500,2.73518,-1.57747,0
17591,500,2.74984,-1.58911,0
17592,500,2.73792,-1.57618,0


In [627]:
m1r08_new = m1r08.sort_values(['x','Ts'],ascending=[True,True])
m1r08_new = m1r08_new.reset_index(drop=True)
m1r08_new

,Ts,x,y,dup
0,6,1.03367,0.03535,0
1,7,1.03367,0.03535,1
2,3,1.03382,0.02717,0
3,6,1.03382,0.02717,1
4,2,1.04370,0.00162,0
...,...,...,...,...
17589,220,5.20485,-0.39341,0
17590,218,5.22241,-0.29380,0
17591,220,5.22775,-0.39173,0
17592,218,5.30332,-0.26446,0


In [628]:
all_l = np.around( m1r08_new[m1r08_new.columns[1:3]], 5).values.tolist()

In [544]:
all_l[0]

[1.03367, 0.03535]

In [545]:
all_l[1]

[1.03367, 0.03535]

In [546]:
all_l[1] == all_l[0]

True

In [558]:
np.absolute(m1r08_new['Ts'][1] - m1r08_new['Ts'][0])

1

In [557]:
m1r08_new['dup'][0]

0

In [637]:
idx = 0

for i in all_l:
    if all_l[idx+1] == all_l[idx]:
        if np.absolute(m1r08_new['Ts'][idx+1] - m1r08_new['Ts'][idx]) == 1: # identify an aggressive motor (label 1.0)
            #print('%s Aggressive motor'%m1r08_new['x'].loc[idx])
            #print('%s Aggressive motor'%m1r08_new['x'].loc[idx+1])
            m1r08_new['dup'].loc[idx] = 1.0
            m1r08_new['dup'].loc[idx+1] = 1.0
        elif np.absolute(m1r08_new['Ts'][idx+1] - m1r08_new['Ts'][idx]) != 1: # identify a persistent disappearing motor (label 0.1)
            try:
                if all_l[idx+1] != all_l[idx+2]:
                    #print('%s Persistent disappearing '%m1r08_new['x'].loc[idx])
                    #print('%s Persistent disappearing '%m1r08_new['x'].loc[idx+1])
                    m1r08_new['dup'].loc[idx] = 0.1
                    m1r08_new['dup'].loc[idx+1] = 0.1
                    #pass
                elif all_l[idx+1] == all_l[idx+2]:
                    if np.absolute(m1r08_new['Ts'][idx+1] - m1r08_new['Ts'][idx+2]) != 1:
                        #print('%s Persistent disappearing '%m1r08_new['x'].loc[idx+1])
                        m1r08_new['dup'].loc[idx+1] = 0.1
                        #pass
                    elif np.absolute(m1r08_new['Ts'][idx+1] - m1r08_new['Ts'][idx+2]) == 1:
                        #print('%s Persistent disappearing '%m1r08_new['x'].loc[idx])
                        m1r08_new['dup'].loc[idx] = 0.1
                        #pass
            except Exception as e:
                if str(e) == 'list index out of range':
                    pass
                else:
                    print('Unknown error!')
    elif all_l[idx+1] != all_l[idx]: # identify a unique disappearing motor (label 0.0)
        try:
            if all_l[idx+1] != all_l[idx+2]:
                #print(m1r08_new['x'].loc[idx+1])
                #print('%s: Unique disappearing motor!'%all_l[idx+1])
                m1r08_new['dup'].loc[idx+1] = 0.0
                #pass
        except Exception as e:
            if str(e) == 'list index out of range':
                #print(m1r08_new['x'].loc[idx+1])
                #print('%s: Unique disappearing motor at the end!'%all_l[idx+1])
                m1r08_new['dup'].loc[idx+1] = 0.0
                #pass
            else:
                print('Unknown error!')
    if idx < (len(all_l)-2):
        idx+=1

In [635]:
m1r08_new.to_csv('x_new.csv', index=False, float_format='%.5f')

In [614]:
n = [[1,2],[3,4]]#,[1,2]]
try:
    if (n[1] != n[0]) and (n[1] != n[2]) == True:
        print('%s: Unique disappearing motor!'%n[1])
except Exception as e:
    if str(e) == 'list index out of range':
        print('%s: Unique disappearing motor at the end!'%n[1])
    else:
        print('Unknown error!')

[3, 4]: Unique disappearing motor at the end!


In [565]:
m1r08_new

,Ts,x,y,dup
0,6,1.03367,0.03535,1
1,7,1.03367,0.03535,1
2,3,1.03382,0.02717,0
3,6,1.03382,0.02717,1
4,2,1.04370,0.00162,0
...,...,...,...,...
17589,220,5.20485,-0.39341,0
17590,218,5.22241,-0.29380,0
17591,220,5.22775,-0.39173,0
17592,218,5.30332,-0.26446,0


In [541]:
m1r08_new['dup'][0]

0

In [575]:
m1r08_new.to_csv('all_active133R08_new.csv', index=False, float_format='%.5f')

In [566]:
m1r08dups = m1r08_new[m1r08_new.dup == 1]
m1r08dups

,Ts,x,y,dup
0,6,1.03367,0.03535,1
1,7,1.03367,0.03535,1
3,6,1.03382,0.02717,1
8,6,1.05077,0.01846,1
9,7,1.05077,0.01846,1
...,...,...,...,...
17573,215,5.13170,-0.26294,1
17575,217,5.13622,-0.31561,1
17578,192,5.13829,-0.38452,1
17579,203,5.13829,-0.38452,1


In [567]:
m1r08dupsx = m1r08dups.drop(['dup'], axis=1)
m1r08dupsx = m1r08dupsx.reset_index(drop=True)
m1r08dupsx

,Ts,x,y
0,6,1.03367,0.03535
1,7,1.03367,0.03535
2,6,1.03382,0.02717
3,6,1.05077,0.01846
4,7,1.05077,0.01846
...,...,...,...
9850,215,5.13170,-0.26294
9851,217,5.13622,-0.31561
9852,192,5.13829,-0.38452
9853,203,5.13829,-0.38452


In [568]:
#m1r08dupsx_ = m1r08dupsx.sort_values('x')
#m1r08dupsx_

In [569]:
#m1r08dupsx_2 = m1r08dupsx.sort_values(['x','Ts'],ascending=[True,True])
#m1r08dupsx_2

In [570]:
#m1r08dup_srtd = m1r08dupsx_2.reset_index(drop=True)
#m1r08dup_srtd

In [571]:
#m1r08dup_srtd.to_csv('m1sort_x_2.csv', index=False, float_format='%.5f')

In [638]:
a = np.around( m1r08dupsx[m1r08dupsx.columns[1:3]], 5).values.tolist()


In [523]:
np.around( m1r08dup_srtd[m1r08dup_srtd.columns[0:3]], 5).values.tolist()[8947]

[217.0, 5.13622, -0.31561]

In [524]:
np.around( m1r08dup_srtd[m1r08dup_srtd.columns[1:3]], 5).values.tolist()[8947]

[5.13622, -0.31561]

In [525]:
np.around( m1r08dup_srtd[m1r08dup_srtd.columns[1:3]].iloc[8947], 5).values.tolist()

[5.13622, -0.31561]

In [585]:
m1r08dup_srtd[m1r08dup_srtd.columns[0]].iloc[8947]

217

In [527]:
np.absolute(m1r08dup_srtd[m1r08dup_srtd.columns[0]].iloc[8949] - m1r08dup_srtd[m1r08dup_srtd.columns[0]].iloc[8949-1])

11

In [587]:
seen = {}
dupes = []
ts_value = []
idx = 0
a = np.around( m1r08dupsx[m1r08dupsx.columns[1:3]], 5).values.tolist()

for x in a:
    if str(x) not in seen:
        seen[str(x)]=0
        ts_value.append(m1r08dupsx[m1r08dupsx.columns[0]].iloc[idx])
    else:
        if np.absolute(m1r08dupsx[m1r08dupsx.columns[0]].iloc[idx] - m1r08dupsx[m1r08dupsx.columns[0]].iloc[idx-1]) == 1:
            seen[str(x)] += 1
            ts_value.append(m1r08dupsx[m1r08dupsx.columns[0]].iloc[idx])
        if seen[str(x)] == 1:
            dupes.append(x)
    idx+=1

In [588]:
seen

{'[1.03367, 0.03535]': 1,
 '[1.03382, 0.02717]': 0,
 '[1.05077, 0.01846]': 1,
 '[1.06578, 0.03027]': 1,
 '[1.076, 0.02888]': 1,
 '[1.11765, 0.00967]': 2,
 '[1.12169, 0.01038]': 0,
 '[1.13077, 0.03635]': 1,
 '[1.13428, 0.02329]': 0,
 '[1.1404, 0.0273]': 2,
 '[1.14359, 0.03007]': 0,
 '[1.15, 0.00922]': 0,
 '[1.15155, 0.03391]': 1,
 '[1.15414, 0.03876]': 0,
 '[1.15647, 0.03387]': 0,
 '[1.18057, 0.02772]': 1,
 '[1.19434, 0.07837]': 1,
 '[1.19571, 0.02284]': 2,
 '[1.20024, 0.02048]': 0,
 '[1.21382, 0.1024]': 0,
 '[1.2227, 0.10223]': 0,
 '[1.22951, 0.0226]': 3,
 '[1.2718, 0.0138]': 0,
 '[1.28024, 0.04462]': 0,
 '[1.28767, 0.02245]': 2,
 '[1.3008, 0.04384]': 2,
 '[1.30572, 0.0157]': 1,
 '[1.31526, 0.10632]': 0,
 '[1.32072, 0.02215]': 0,
 '[1.33737, 0.02066]': 0,
 '[1.3424, 0.03051]': 0,
 '[1.34254, 0.01794]': 0,
 '[1.35155, 0.03508]': 1,
 '[1.35313, 0.00908]': 0,
 '[1.35478, 0.01689]': 1,
 '[1.35481, 0.08481]': 0,
 '[1.35505, 0.03543]': 1,
 '[1.3559, 0.02964]': 2,
 '[1.36969, 0.02999]': 1,
 '

In [465]:
m1r08dups.to_csv('all_active_dup133R08.csv', index=False, float_format='%.5f')

In [533]:
test = pd.DataFrame([[1,2,3],[4,5,6],[1,2,3],[4,8,7],[1,2,3],[4,5,6]], columns=['x','y','z'])
test

,x,y,z
0,1,2,3
1,4,5,6
2,1,2,3
3,4,8,7
4,1,2,3
5,4,5,6


In [592]:
test['z'].loc[3] = 12
test

,x,y,z
0,1,2,3
1,4,5,6
2,1,2,3
3,4,8,12
4,1,2,11
5,4,5,6


In [282]:
test2 = pd.DataFrame([[4,8,7],[1,2,3],[4,5,6],[9,9,9]], columns=['x','y','z'])
test2

,x,y,z
0,4,8,7
1,1,2,3
2,4,5,6
3,9,9,9


In [82]:
for i in test.values:
    print(i)

[1 2 3]
[4 5 6]
[1 2 3]
[4 8 7]
[1 2 3]
[4 5 6]


In [83]:
for x in test.values:
    print(x)

[1 2 3]
[4 5 6]
[1 2 3]
[4 8 7]
[1 2 3]
[4 5 6]


In [180]:
test[test.duplicated() == True]

,x,y,z
2,1,2,3
4,1,2,3
5,4,5,6


In [181]:
np.sum(test.duplicated())

3

In [182]:
test

,x,y,z
0,1,2,3
1,4,5,6
2,1,2,3
3,4,8,7
4,1,2,3
5,4,5,6


In [284]:
test3 = test.append(test2)
test3

,x,y,z
0,1,2,3
1,4,5,6
2,1,2,3
3,4,8,7
4,1,2,3
5,4,5,6
0,4,8,7
1,1,2,3
2,4,5,6
3,9,9,9


In [220]:
test3.duplicated()

0    False
1    False
2     True
3    False
4     True
5     True
0     True
1     True
2     True
dtype: bool

In [227]:
if ([1,2,3] == np.array([[1,2,3],[4,5,6],[1,2,3]])).all(1).any() == True:
    print('duplicate(s) exist')

duplicate(s) exist


In [212]:
idx = 1
dup_dict = {}
seen = []

for i in test3[test3.duplicated() == True].values:
    print(i)
    if idx not in dup_dict.keys():
        dup_dict.update({idx:[i.tolist()]})
    else:
        dup_dict[idx].append(i.tolist())

[1 2 3]
[1 2 3]
[4 5 6]
[4 8 7]
[1 2 3]
[4 5 6]


In [214]:
dup_dict

{1: [[1, 2, 3], [1, 2, 3], [4, 5, 6], [4, 8, 7], [1, 2, 3], [4, 5, 6]]}

In [195]:
{0:[[1,2,3],[1,4,1]], 2:[1,8,7]}

{0: [[1, 2, 3], [1, 4, 1]], 2: [1, 8, 7]}

In [275]:
seen = np.array([[0,0,0]])
seen

array([[0, 0, 0]])

In [276]:
elem = np.array([i.tolist()])
elem

array([[4, 5, 6]])

In [277]:
np.concatenate(( seen,elem ))

array([[0, 0, 0],
       [4, 5, 6]])

In [641]:
idx = 1
dup_dict = {}
seen = np.array([[0,0,0]])
buc = []

for i in test3[test3.duplicated() == True].values:
    #print(i.tolist())
    elem = np.array([i.tolist()])#; print('elem %s'%elem)
    if (i.tolist() == seen).all(1).any() == True:
        #print('duplicate(s) seen before')
        pass
    else:
        if idx not in dup_dict.keys():
            #print('new Ts')
            dup_dict.update({idx:[i.tolist()]})
            seen = np.concatenate(( seen,elem ))
        else:
            #print('old Ts %s'%[i.tolist()])
            dup_dict[idx].append(i.tolist())
            seen = np.concatenate(( seen,elem ))
    #print(seen)
    #print(dup_dict)

In [287]:
dup_dict

{1: [[1, 2, 3], [4, 5, 6], [4, 8, 7]]}

In [642]:
# Extreamly slow and not confirmed. Not used.

idx = 1
dup_dict = {}
freq_dict = {}
seen = np.array([[0,0,0]])
arr = m1[0]
dupNo = []
#idx = 0

for idx in range(Ts):
    #print(idx)
    dups = np.sum(arr.duplicated())
    dupNo.append(dups)
    for i in arr[arr.duplicated() == True].values:
        #print(i.tolist())
        ix = str(np.around(i.tolist(),5))
        if ix not in freq_dict:
            freq_dict[ix]=1
        else:
            freq_dict[ix] += 1
        elem = np.array([i.tolist()])#; print('elem %s'%elem)
        if (i.tolist() == seen).all(1).any() == True:
            #print('duplicate(s) seen before')
            pass
        else:
            if idx not in dup_dict.keys():
                #print('new Ts')
                dup_dict.update({idx:[i.tolist()]})
                seen = np.concatenate(( seen,elem ))
            else:
                #print('old Ts %s'%[i.tolist()])
                dup_dict[idx].append(i.tolist())
                seen = np.concatenate(( seen,elem ))
        #print(seen)
        #print(dup_dict)
    if idx < Ts-1:
        arr = arr.append(m1[idx+1])

In [383]:
arr.to_csv('allAbm.csv', index=True, float_format='%.5f')

In [340]:
sb = {}
sb

{}

In [341]:
i.tolist()

[2.9346099999999997, -1.46986, 0.0]

In [342]:
sb[str(np.around(i.tolist(),5))] = 1
sb

{'[ 2.93461 -1.46986  0.     ]': 1}

In [344]:
sb[str(np.around(i.tolist(),5))] += 1
sb

{'[ 2.93461 -1.46986  0.     ]': 3}

In [347]:
x = str(np.around(i.tolist(),5))
if x not in sb:
    sb[x]=1
else:
    sb[x] += 1

In [348]:
sb

{'[ 2.93461 -1.46986  0.     ]': 5}

In [318]:
sb.values()

dict_values([[1, 2, 3]])

In [172]:
arr = test
dupNo = []

for i in range(2):
    dups = np.sum(test.duplicated())
    dupNo.append(dups)
    test = test.append(test2)

In [112]:
([1,2,3] == test.values).all(1)#.any() 

array([ True, False,  True, False,  True, False])

In [166]:
npi.contains(test.values, test2.values)

array([ True,  True,  True])

In [ ]:
seen = {}
dupes = []

for x in test.values:
    if x not in seen:
        seen[x]=1
    else:
        if seen[x] == 1:
            dupes.append(x)
        seen[x] += 1

In [47]:
np.in1d([6,4,5], test.values)

array([ True,  True,  True])